In [1]:
# todo
# 修改info代码 gflops task.py self.info()
# ! 测试coco预训练的p2模型的精度 并和标准v8n对比

# 组合1: 1280+lsk+aug
# 组合2: 1280+p2+att+aug (p2+att 全部跑一遍 选一个最好的) p2选好att后 最优的模型再选fpn 再选iou 

# 在bars_runway的测试集进行测试
# bbox融合可以作为 冗余备份
# 接近跑道 runway detection -> fod 
# aug 分多类 实验

In [1]:
import os
import glob
import numpy as np
import pandas as pd

# metrics文件路径定义
path_baseline_orin = 'results/tables/metrics/baseline/orin'
path_baseline_1280 = 'results/tables/metrics/baseline/1280'
path_yolov8_640 = 'results/tables/metrics/yolov8_640'
path_yolov8_1280 = 'results/tables/metrics/yolov8_1280'
path_finetune = 'results/tables/metrics/finetune'

# baseline orin & 1280
all_path_csv_baseline_orin = glob.glob(f'{path_baseline_orin}/*.csv')
all_path_csv_baseline_1280 = glob.glob(f'{path_baseline_1280}/*.csv')

# yolov8 640
all_path_csv_yolov8_640 = glob.glob(f'{path_yolov8_640}/*.csv')

# yolov8 1280 
all_path_csv_yolov8_1280 = glob.glob(f'{path_yolov8_1280}/*.csv')
all_path_csv_yolov8_base_1280 = [path for path in all_path_csv_yolov8_1280 if 'base' in path]
all_path_csv_yolov8_att_1280 = [path for path in all_path_csv_yolov8_1280 if 'att' in path]
all_path_csv_yolov8_tf_1280 = [path for path in all_path_csv_yolov8_1280 if 'tf' in path]
all_path_csv_yolov8_aug_1280 = [path for path in all_path_csv_yolov8_1280 if 'aug' in path and 'merge' not in path]
all_path_csv_yolov8_iou_1280 = [path for path in all_path_csv_yolov8_1280 if 'iou' in path]
all_path_csv_yolov8_neck_1280 = [path for path in all_path_csv_yolov8_1280 if 'neck' in path]
all_path_csv_yolov8_up_1280 = [path for path in all_path_csv_yolov8_1280 if 'up' in path]
all_path_csv_yolov8_merge_1280 = [path for path in all_path_csv_yolov8_1280 if 'merge' in path and 'aug' not in path]
all_path_csv_proposed_1280 = [path for path in all_path_csv_yolov8_1280 if 'yolov8n-p2_lsk_bifpn_DIoU' in path]

# yolov8 1280 finetune
all_path_csv_finetune = glob.glob(f'{path_finetune}/*.csv')

type_test = 'best'  # 目前写死best不改动

# 关键模型metrics
path_base_yolov8n_640 = f'results/tables/metrics/yolov8_640/metrics_{type_test}_base_yolov8n.csv'
path_base_yolov8n_p2_640 = f'results/tables/metrics/yolov8_640/metrics_{type_test}_base_yolov8n-p2.csv'
path_base_yolov8n_1280 = f'results/tables/metrics/yolov8_1280/metrics_{type_test}_base_yolov8n_1280.csv'
path_base_yolov8n_p2_1280 = f'results/tables/metrics/yolov8_1280/metrics_{type_test}_base_yolov8n-p2_1280.csv'
path_proposed_1280 = f'results/tables/metrics/yolov8_1280/metrics_{type_test}_merge_yolov8n-p2_lsk_bifpn_DIoU_aug10_1280.csv'

In [35]:
def get_model_name(filename):
    if 'finetune' not in filename:
        return filename.split(f'metrics_')[1].split('.csv')[0]
    else:
        return filename.split(f'metrics_last_')[1].split('.csv')[0]

def get_metrics(path_csv, type_map='5095'):
    df = pd.read_csv(path_csv)
    if type_map == '5095':
        row = df.iloc[0:1].copy()
    elif type_map == '50':
        row = df.iloc[1:2].copy()
    
    if df.shape[0]>12:
        row['param'] = df.iloc[12,1]
        row['flops'] = df.iloc[13,1]
        row['fps'] = df.iloc[15,1:].mean().round(1)
        
    row['metrics'] = get_model_name(os.path.basename(path_csv))
    return row

def get_size_map(path_csv):
    size_map = pd.read_csv(path_csv, index_col='metrics').iloc[3:6,:]
    return size_map

def set_metrics(metrics, sort=True):
    metrics = pd.concat(metrics, axis=0)
    metrics.rename(columns={'metrics': 'models'}, inplace=True)
    metrics.set_index('models', inplace=True)
    if sort:
        metrics.sort_values(by='test_real', ascending=True, inplace=True)
    return metrics

def display_metircs(metrics, basemodel):
    base_index = metrics.index.get_loc(basemodel)
    metrics = metrics.iloc[base_index:]
    
    columns = metrics.columns.tolist()
    if len(columns) > 4:
        display(metrics.style.bar(align='left', subset=columns[:-3]).format("{:.3f}", subset=columns[:-1]).format("{:.1f}", subset=columns[-1]))
    else:
        display(metrics.style.bar(align='left').format("{:.3f}"))

def get_formatted_improve(df2, improve_percent):
    improve_percent_formatted = improve_percent.applymap(lambda x: f"({'+' if x >= 0 else '-'}{abs(x):04.1f}%)")  # 补0 追加sign和()
    df2 = df2.applymap(lambda x: f"{x:.3f}")
    df_concatenated = df2 + " " + improve_percent_formatted
    return df_concatenated


# 基模型

In [44]:
all_path_300 = glob.glob('results/tables/metrics/*.csv')
# baseline
metrics = [get_metrics(path) for path in all_path_300]
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640|att_yolov8n|tf_yolov8n')]
display_metircs(metrics, 'base_yolov8n_640')

,test_synth,test_real_nominal,test_real_edge,test_real
models,,,,
base_yolov8n_640,0.754,0.586,0.348,0.545
tf_yolov8n_vit_640,0.746,0.583,0.358,0.545
att_yolov8n_cpca_640,0.760,0.588,0.358,0.549
att_yolov8n_ese_640,0.758,0.588,0.372,0.551
att_yolov8n_ta_640,0.760,0.590,0.366,0.552
att_yolov8n_lska_640,0.751,0.590,0.366,0.552
att_yolov8n_sa_640,0.766,0.592,0.381,0.556


In [39]:
all_path_300 = glob.glob('results/tables/metrics/*.csv')
# baseline
metrics = [get_metrics(path) for path in all_path_300]
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640|neck_yolov8n')]
display_metircs(metrics, 'base_yolov8n_640')

,test_synth,test_real_nominal,test_real_edge,test_real
models,,,,
base_yolov8n_640,0.754,0.586,0.348,0.545
neck_yolov8n_bifpn_640,0.758,0.591,0.365,0.553


In [41]:
all_path_300 = glob.glob('results/tables/metrics/*.csv')
# baseline
metrics = [get_metrics(path) for path in all_path_300]
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640|iou_yolov8n')]
display_metircs(metrics, 'base_yolov8n_640')

,test_synth,test_real_nominal,test_real_edge,test_real
models,,,,
base_yolov8n_640,0.754,0.586,0.348,0.545
iou_yolov8n_ShapeIoU_640,0.761,0.582,0.369,0.545
iou_yolov8n_MDPIoU1_640,0.746,0.589,0.357,0.549
iou_yolov8n_GIoU_640,0.765,0.588,0.371,0.551
iou_yolov8n_MDPIoU2_640,0.759,0.593,0.380,0.557
iou_yolov8n_DIoU_640,0.765,0.594,0.383,0.558
iou_yolov8n_SIoU_640,0.761,0.601,0.382,0.563


In [43]:
all_path_300 = glob.glob('results/tables/metrics/*.csv')
# baseline
metrics = [get_metrics(path) for path in all_path_300]
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640|aug_yolov8n')]
display_metircs(metrics, 'base_yolov8n_640')

,test_synth,test_real_nominal,test_real_edge,test_real
models,,,,
base_yolov8n_640,0.754,0.586,0.348,0.545
aug_yolov8n_aug_color_blur_geo_weather_5_640,0.749,0.600,0.383,0.556
aug_yolov8n_aug_color_blur_geo_weather_1_640,0.746,0.595,0.369,0.558
aug_yolov8n_aug_color_blur_geo_weather_20_640,0.755,0.591,0.418,0.561
aug_yolov8n_aug_color_blur_geo_weather_10_640,0.756,0.604,0.413,0.572


In [ ]:
# baseline
metrics = [get_metrics(path) for path in all_path_csv_baseline_orin]

# base_yolov8n_640
metrics.append(get_metrics(path_base_yolov8n_640))

# proposed_1280
# metrics.append(get_metrics(path_proposed_1280))

# DF格式调整
metrics = set_metrics(metrics)

# 过滤打不过的模型
metrics = metrics[metrics.index != 'baseline_centernet_512']
metrics = metrics[metrics.index != 'baseline_retinanet_1333']

# 结果分析
print('证明yolov8模型足够优异, 在各测试集上均吊打其余模型, 因此选择其作为核心模型架构')

# DF样式设计
display_metircs(metrics)

# 增加分辨率

In [5]:
metrics = []

# base_yolov8n_640
metrics.append(get_metrics(path_base_yolov8n_640))

# ! 补充640训练 1280测试的精度

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# DF格式调整
metrics = set_metrics(metrics)

# 结果分析
percent = ((metrics.values[-1,:] - metrics.values[0,:] ) / metrics.values[0,:] * 100).round(3)
print(F"增加图像尺寸显著提升精度(改进{', '.join([f'{i}%' for i in percent])})")
print('证明增大输入图像尺寸有效, 能够显著改善在各测试集上的精度, 因此后续实验选用1280作为输入尺寸')

# DF样式设计
display_metircs(metrics)

增加图像尺寸显著提升精度(改进13.485%, 22.279%, 7.754%, 20.652%, 0.0%, 300.012%, -33.803%)
证明增大输入图像尺寸有效, 能够显著改善在各测试集上的精度, 因此后续实验选用1280作为输入尺寸


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8n,0.749,0.588,0.374,0.552,3.011,8.194,134.9
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3


# 增加P2

In [6]:
metrics = []

# base_yolov8n_640
metrics.append(get_metrics(path_base_yolov8n_640))

# base_yolov8n-p2_640
# metrics.append(get_metrics(path_base_yolov8n_p2_640))

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# base_yolov8n-p2_1280
metrics.append(get_metrics(path_base_yolov8n_p2_1280))

# DF格式调整
metrics = set_metrics(metrics)

# 结果分析
index_base = np.where(metrics.index == f'base_yolov8n_1280')[0][0]
percent = ((metrics.values[-1,:] - metrics.values[index_base,:] ) / metrics.values[index_base,:] * 100).round(3)
print(f"增加P2在synth精度接近的情况下(改进{percent[0]}%)")
print(f"在real精度有显著改进(改进{', '.join([f'{i}%' for i in percent[1:]])}) ")
print('证明增加小尺寸检测头有效, 有助于改善检测精度')

# DF样式设计
display_metircs(metrics)

增加P2在synth精度接近的情况下(改进-0.588%)
在real精度有显著改进(改进3.199%, 7.692%, 3.303%, -2.79%, 50.816%, -8.287%) 
证明增加小尺寸检测头有效, 有助于改善检测精度


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8n,0.749,0.588,0.374,0.552,3.011,8.194,134.9
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3
base_yolov8n-p2_1280,0.845,0.742,0.434,0.688,2.927,49.433,81.9


In [7]:
# 细节分析
df1 = get_size_map(path_base_yolov8n_1280)
df2 = get_size_map(path_base_yolov8n_p2_1280)

improve_percent = ((df2-df1)/df1*100).round(1)

print(
    f'''具体的: 
    test_synth中无small目标, medium精度略微下降({improve_percent.values[1,0]}%), (误差允许情况下认为基本一致), 而large精度保持一致
    rea上, 对small样本的检测精度显著改善({', '.join([f'{i}%' for i in improve_percent.values[0,1:]])}), 而medium和large精度也有较大改善
    '''
)

df2 = get_formatted_improve(df2, improve_percent)
pd.concat([df1.applymap(lambda x: f"{x:.3f}"), df2], axis=0).replace('-1.000', '-').replace('-1.000 (+00.0%)', '-')

具体的: 
    test_synth中无small目标, medium精度略微下降(-2.6%), (误差允许情况下认为基本一致), 而large精度保持一致
    rea上, 对small样本的检测精度显著改善(23.9%, 39.1%, 24.4%), 而medium和large精度也有较大改善
    


,test_synth,test_real_nominal,test_real_edge,test_real
metrics,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.310,0.184,0.291
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.760,0.643,0.347,0.602
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.892,0.871,0.476,0.792
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.384 (+23.9%),0.256 (+39.1%),0.362 (+24.4%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.740 (-02.6%),0.663 (+03.1%),0.354 (+02.0%),0.620 (+03.0%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.892 (+00.0%),0.883 (+01.4%),0.516 (+08.4%),0.812 (+02.5%)


# 使用AUG

In [8]:
metrics = []

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# yolov8_aug_1280
for path in all_path_csv_yolov8_aug_1280:
    metrics.append(get_metrics(path))

# DF格式调整
metrics = set_metrics(metrics)

# 结果分析
index_base = np.where(metrics.index == f'base_yolov8n_1280')[0][0]
percent = ((metrics.values[-1,:] - metrics.values[index_base,:] ) / metrics.values[index_base,:] * 100).round(3)
print(f"在synth精度接近的情况下(改进{percent[0]}%)")
print(f"在real精度有显著改进(改进{', '.join([f'{i}%' for i in percent[1:]])}) ")
print("证明使用数据增强有效, 能显著改进real下的检测精度, 尤其是edge数据")

# DF样式设计
display_metircs(metrics)

在synth精度接近的情况下(改进-0.235%)
在real精度有显著改进(改进4.312%, 36.476%, 7.808%, 0.0%, 0.0%, 5.151%) 
证明使用数据增强有效, 能显著改进real下的检测精度, 尤其是edge数据


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3
aug_yolov8n_aug5_1280,0.853,0.750,0.502,0.709,3.011,32.777,91.5
aug_yolov8n_aug10_1280,0.848,0.750,0.550,0.718,3.011,32.777,93.9


In [9]:
# 细节分析
df1 = get_size_map(path_base_yolov8n_1280)

path = f'results/tables/metrics/yolov8_1280/metrics_{type_test}_aug_yolov8n_aug10_1280.csv'
df2 = get_size_map(path)

improve_percent = ((df2-df1)/df1*100).round(3)
print(
f'''
具体的: 
test_synth中无small目标, large精度略微下降({improve_percent.values[2,0]:.3f}%), (误差允许情况下认为基本一致), 而large精度保持一致
real上, 对small样本的检测精度显著改善({', '.join([f'{i}%' for i in improve_percent.values[0,1:]])}), 而medium和large精度也有较大改善
'''
)

display(df1)
display(df2)
print('改进百分比')
display(improve_percent)


具体的: 
test_synth中无small目标, large精度略微下降(-0.673%), (误差允许情况下认为基本一致), 而large精度保持一致
real上, 对small样本的检测精度显著改善(23.871%, 86.413%, 29.553%), 而medium和large精度也有较大改善



,test_synth,test_real_nominal,test_real_edge,test_real
metrics,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-1.000,0.310,0.184,0.291
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.760,0.643,0.347,0.602
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.892,0.871,0.476,0.792


,test_synth,test_real_nominal,test_real_edge,test_real
metrics,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-1.000,0.384,0.343,0.377
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.764,0.672,0.482,0.645
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.886,0.895,0.636,0.846


改进百分比


,test_synth,test_real_nominal,test_real_edge,test_real
metrics,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-0.000,23.871,86.413,29.553
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.526,4.510,38.905,7.143
"AP (IoU=0.50:0.95, area=large, maxDets=100)",-0.673,2.755,33.613,6.818


# 使用UP


In [10]:
metrics = []

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# yolov8_up_1280
for path in all_path_csv_yolov8_up_1280:
    metrics.append(get_metrics(path))

# DF格式调整
metrics = set_metrics(metrics)

# 结果分析
index_base = np.where(metrics.index == f'base_yolov8n_1280')[0][0]
percent = ((metrics.values[-1,:] - metrics.values[index_base,:] ) / metrics.values[index_base,:] * 100).round(3)
print(f"在synth精度接近的情况下(改进{percent[0]}%)")
print(f"在real精度有显著改进(改进{', '.join([f'{i}%' for i in percent[1:]])}) ")
print("证明使用上采样模块有效, 能显著改进real下的检测精度, 尤其是edge数据")

# DF样式设计
display_metircs(metrics)

在synth精度接近的情况下(改进-0.235%)
在real精度有显著改进(改进1.808%, 4.467%, 1.952%, 2.391%, 2.175%, -4.591%) 
证明使用上采样模块有效, 能显著改进real下的检测精度, 尤其是edge数据


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3
up_yolov8n_carafe_1280,0.848,0.732,0.421,0.679,3.083,33.490,85.2


# 使用ATT

In [11]:
metrics = []

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# yolov8_att_1280
for path in all_path_csv_yolov8_att_1280:
    metrics.append(get_metrics(path))

# yolov8_tf_1280
for path in all_path_csv_yolov8_tf_1280:
    metrics.append(get_metrics(path))

# DF格式调整
metrics = set_metrics(metrics)

# 过滤打不过的模型
# metrics = metrics[metrics.index != 'att_yolov8n_ese_1280']
# metrics = metrics[metrics.index != 'att_yolov8n_gam_1280']
# metrics = metrics[metrics.index != 'att_yolov8n_cpca_1280']

# 结果分析
index_base = np.where(metrics.index == 'base_yolov8n_1280')[0][0]
percent = ((metrics.values[-1,:] - metrics.values[index_base,:] ) / metrics.values[index_base,:] * 100).round(3)
print(f"增加ATT精度有显著改进(改进{', '.join([f'{i}%' for i in percent])}) ")
print("证明使用LSK有效, 能显著改善在各测试集的检测精度")

# DF样式设计
display_metircs(metrics)

增加ATT精度有显著改进(改进-0.353%, 3.616%, 12.655%, 4.354%, 2.192%, 0.644%, -0.784%) 
证明使用LSK有效, 能显著改善在各测试集的检测精度


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
att_yolov8n-p2_cbam_1280,0.844,0.705,0.404,0.654,2.993,49.645,82.2
att_yolov8n_cpca_1280,0.848,0.721,0.389,0.664,3.138,33.700,90.9
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3
att_yolov8n_se_1280,0.848,0.727,0.391,0.668,3.019,32.804,88.0
att_yolov8n_eca_1280,0.842,0.722,0.418,0.671,3.011,32.782,90.6
att_yolov8n-p2_se_1280,0.851,0.724,0.412,0.673,2.935,49.461,77.8
att_yolov8n_ema_1280,0.843,0.729,0.419,0.676,3.012,32.860,89.8
tf_yolov8n_vit_1280,0.846,0.729,0.431,0.678,4.849,36.132,68.9
att_yolov8n_lska_1280,0.847,0.732,0.410,0.679,3.089,33.022,94.4


In [12]:
# 细节分析
df1 = get_size_map(path_base_yolov8n_1280)

path = f'results/tables/metrics/yolov8_1280//metrics_{type_test}_att_yolov8n_lsk_1280.csv'
df2 = get_size_map(path)

improve_percent = ((df2-df1)/df1*100).round(1)
print(
    f'''具体的: 
    test_synth上性能保持一致
    real上, 对small样本的检测精度显著改善({', '.join([f'{i}%' for i in improve_percent.values[0,1:]])}), 而medium和large精度也有较大改善
    '''
)

df2 = get_formatted_improve(df2, improve_percent)
pd.concat([df1.applymap(lambda x: f"{x:.3f}"), df2], axis=0).replace('-1.000', '-').replace('-1.000 (+00.0%)', '-')

具体的: 
    test_synth上性能保持一致
    real上, 对small样本的检测精度显著改善(23.2%, 30.4%, 23.4%), 而medium和large精度也有较大改善
    


,test_synth,test_real_nominal,test_real_edge,test_real
metrics,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.310,0.184,0.291
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.760,0.643,0.347,0.602
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.892,0.871,0.476,0.792
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.382 (+23.2%),0.240 (+30.4%),0.359 (+23.4%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.760 (+00.0%),0.679 (+05.6%),0.395 (+13.8%),0.638 (+06.0%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.892 (+00.0%),0.874 (+00.3%),0.522 (+09.7%),0.807 (+01.9%)


# 替换IOU

In [13]:
metrics = []

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# yolov8_iou_1280
for path in all_path_csv_yolov8_iou_1280:
    metrics.append(get_metrics(path))

# DF格式调整
metrics = set_metrics(metrics)

# 过滤打不过的模型
metrics = metrics[metrics.index != 'iou_yolov8n_NWD_1280']
metrics = metrics[metrics.index != 'iou_yolov8n_WIoU_1280']

# 结果分析
index_base = np.where(metrics.index == 'base_yolov8n_1280')[0][0]
percent = ((metrics.values[-1,:] - metrics.values[index_base,:] ) / metrics.values[index_base,:] * 100).round(3)
print(f"替换IOU在synth精度一致的情况下(改进{percent[0]}%)")
print(f"在real精度有显著改进(改进{', '.join([f'{i}%' for i in percent[1:]])}) ")
print("证明使用DIoU有效, 同时修改损失函数和TAL得分函数, 能显著改善在各测试集的检测精度")

# DF样式设计
display_metircs(metrics)

替换IOU在synth精度一致的情况下(改进0.235%)
在real精度有显著改进(改进3.199%, 10.67%, 3.904%, 0.0%, 0.0%, 7.055%) 
证明使用DIoU有效, 同时修改损失函数和TAL得分函数, 能显著改善在各测试集的检测精度


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3
iou_yolov8n_EIoU_1280,0.843,0.722,0.412,0.668,3.011,32.777,94.8
iou_yolov8n_MDPIoU1_1280,0.816,0.730,0.440,0.682,3.011,32.777,94.1
iou_yolov8n_MDPIoU2_1280,0.847,0.734,0.429,0.683,3.011,32.777,97.1
iou_yolov8n_GIoU_1280,0.845,0.735,0.432,0.684,3.011,32.777,94.8
iou_yolov8n_ShapeIoU_1280,0.849,0.740,0.434,0.687,3.011,32.777,96.4
iou_yolov8n_SIoU_1280,0.845,0.739,0.422,0.687,3.011,32.777,94.3
iou_yolov8n_DIoU_1280,0.852,0.742,0.446,0.692,3.011,32.777,95.6


In [14]:
# 细节分析
df1 = get_size_map(path_base_yolov8n_1280)
path = f'results/tables/metrics/yolov8_1280//metrics_{type_test}_iou_yolov8n_DIoU_1280.csv'
df2 = get_size_map(path)

improve_percent = ((df2-df1)/df1*100).round(1)
print(
    f'''具体的: 
    test_synth中无small目标, large精度略有提升, medium略有下降(误差允许情况下认为基本一致)
    real上, 对small样本的检测精度显著改善({', '.join([f'{i}%' for i in improve_percent.values[0,1:]])}), 而medium和large精度也有较大改善
    '''
)

df2 = get_formatted_improve(df2, improve_percent)
pd.concat([df1.applymap(lambda x: f"{x:.3f}"), df2], axis=0).replace('-1.000', '-').replace('-1.000 (+00.0%)', '-')

具体的: 
    test_synth中无small目标, large精度略有提升, medium略有下降(误差允许情况下认为基本一致)
    real上, 对small样本的检测精度显著改善(24.5%, 77.2%, 28.5%), 而medium和large精度也有较大改善
    


,test_synth,test_real_nominal,test_real_edge,test_real
metrics,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.310,0.184,0.291
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.760,0.643,0.347,0.602
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.892,0.871,0.476,0.792
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.386 (+24.5%),0.326 (+77.2%),0.374 (+28.5%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.755 (-00.7%),0.668 (+03.9%),0.434 (+25.1%),0.635 (+05.5%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.893 (+00.1%),0.881 (+01.1%),0.479 (+00.6%),0.803 (+01.4%)


# 使用Neck

In [15]:
metrics = []

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# yolov8_neck_1280
for path in all_path_csv_yolov8_neck_1280:
    metrics.append(get_metrics(path))

# DF格式调整
metrics = set_metrics(metrics)

# 过滤打不过的模型
metrics = metrics[metrics.index != 'neck_yolov8n_afpn_small_1280']

# 结果分析
index_base = np.where(metrics.index == 'base_yolov8n_1280')[0][0]
percent = ((metrics.values[-1,:] - metrics.values[index_base,:] ) / metrics.values[index_base,:] * 100).round(3)
print(f"改进Neck在synth精度接近的情况下(改进{percent[0]}%)")
print(f"在real精度有显著改进(改进{', '.join([f'{i}%' for i in percent[1:]])}) ")
print('AFPN模型复杂度较高 该数据集难以重复训练')

# DF样式设计
display_metircs(metrics)

改进Neck在synth精度接近的情况下(改进-0.471%)
在real精度有显著改进(改进2.921%, 11.663%, 3.604%, 0.531%, 0.638%, -2.352%) 
AFPN模型复杂度较高 该数据集难以重复训练


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3
neck_yolov8n_afpn_1280,0.854,0.739,0.411,0.684,4.667,62.562,75.3
neck_yolov8n_bifpn_1280,0.846,0.740,0.450,0.690,3.027,32.986,87.2


In [16]:
# 细节分析
df1 = get_size_map(path_base_yolov8n_1280)
path = f'results/tables/metrics/yolov8_1280/metrics_{type_test}_neck_yolov8n_bifpn_1280.csv'
df2 = get_size_map(path)

improve_percent = ((df2-df1)/df1*100).round(1)
print(
    f'''具体的: 
    test_synth中无small目标, large和medium略有下降(误差允许情况下认为基本一致)
    real上, 对small样本的检测精度显著改善({', '.join([f'{i}%' for i in improve_percent.values[0,1:]])}), 而medium和large精度也有较大改善
    '''
)

df2 = get_formatted_improve(df2, improve_percent)
pd.concat([df1.applymap(lambda x: f"{x:.3f}"), df2], axis=0).replace('-1.000', '-').replace('-1.000 (+00.0%)', '-')

具体的: 
    test_synth中无small目标, large和medium略有下降(误差允许情况下认为基本一致)
    real上, 对small样本的检测精度显著改善(21.0%, 60.3%, 24.7%), 而medium和large精度也有较大改善
    


,test_synth,test_real_nominal,test_real_edge,test_real
metrics,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.310,0.184,0.291
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.760,0.643,0.347,0.602
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.892,0.871,0.476,0.792
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.375 (+21.0%),0.295 (+60.3%),0.363 (+24.7%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.758 (-00.3%),0.673 (+04.7%),0.406 (+17.0%),0.636 (+05.6%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.887 (-00.6%),0.875 (+00.5%),0.505 (+06.1%),0.803 (+01.4%)


# 消融实验

In [17]:
path_ablation = [

    path_base_yolov8n_p2_1280,
    'results/tables/metrics/yolov8_1280/metrics_best_neck_yolov8n_bifpn_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_att_yolov8n_lsk_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_iou_yolov8n_DIoU_1280.csv',

    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n-p2_bifpn_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n-p2_lsk_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n-p2_DIoU_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n_lsk_bifpn_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n_bifpn_DIoU_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n_lsk_DIoU_1280.csv',

    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n-p2_lsk_bifpn_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n-p2_bifpn_DIoU_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n-p2_lsk_DIoU_1280.csv',
    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n_lsk_bifpn_DIoU_1280.csv',

    'results/tables/metrics/yolov8_1280/metrics_best_merge_yolov8n-p2_lsk_bifpn_DIoU_1280.csv',
]

In [18]:
metrics = []

# base_yolov8n_640
metrics.append(get_metrics(path_base_yolov8n_640))

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# yolov8_merge_1280
# for path in all_path_csv_yolov8_merge_1280:
for path in path_ablation:
    metrics.append(get_metrics(path))

# DF格式调整
metrics = set_metrics(metrics, sort=False)

# 过滤打不过的模型
metrics = metrics[metrics.index != 'merge_yolov8n-p2_afpn_small_1280']
# metrics = metrics[metrics.index != 'att_yolov8n_lsk_1280']

# 结果分析
index_base = np.where(metrics.index == 'base_yolov8n_1280')[0][0]
percent = ((metrics.values[-1,:] - metrics.values[index_base,:] ) / metrics.values[index_base,:] * 100).round(3)
print(f"提出模型在精度有显著改进(改进{', '.join([f'{i}%' for i in percent])})")

# DF样式设计
display_metircs(metrics)

提出模型在精度有显著改进(改进-0.118%, 3.199%, 8.189%, 3.904%, 1.827%, 53.242%, -14.67%)


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8n,0.749,0.588,0.374,0.552,3.011,8.194,134.9
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3
base_yolov8n-p2_1280,0.845,0.742,0.434,0.688,2.927,49.433,81.9
neck_yolov8n_bifpn_1280,0.846,0.740,0.450,0.690,3.027,32.986,87.2
att_yolov8n_lsk_1280,0.852,0.744,0.449,0.694,3.130,33.152,92.7
iou_yolov8n_DIoU_1280,0.852,0.742,0.446,0.692,3.011,32.777,95.6
merge_yolov8n-p2_bifpn_1280,0.845,0.738,0.434,0.686,2.947,49.852,77.9
merge_yolov8n-p2_lsk_1280,0.843,0.732,0.420,0.677,3.045,49.809,81.2
merge_yolov8n-p2_DIoU_1280,0.842,0.738,0.411,0.681,2.927,49.433,80.4


# Proposed数据增强

In [19]:
metrics = []

# base_yolov8n_1280
metrics.append(get_metrics(path_base_yolov8n_1280))

# proposed_1280
for path in all_path_csv_proposed_1280:
    metrics.append(get_metrics(path))

# DF格式调整
metrics = set_metrics(metrics)

# 过滤打不过的模型
metrics = metrics[metrics.index != 'merge_yolov8n-p2_lsk_bifpn_DIoU_aug15_1280']
metrics = metrics[metrics.index != 'merge_yolov8n-p2_lsk_bifpn_DIoU_aug20_1280']

# 结果分析
index_base = np.where(metrics.index == 'base_yolov8n_1280')[0][0]
percent = ((metrics.values[-1,:] - metrics.values[index_base,:] ) / metrics.values[index_base,:] * 100).round(3)
print(f"提出模型在精度有显著改进(改进{', '.join([f'{i}%' for i in percent])})")

# DF样式设计
display_metircs(metrics)

提出模型在精度有显著改进(改进0.706%, 6.398%, 31.514%, 9.309%, 1.827%, 53.242%, -12.206%)


,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8n_1280,0.850,0.719,0.403,0.666,3.011,32.777,89.3
merge_yolov8n-p2_lsk_bifpn_DIoU_1280,0.849,0.742,0.436,0.692,3.066,50.228,76.2
merge_yolov8n-p2_lsk_bifpn_DIoU_aug1_1280,0.853,0.750,0.456,0.702,3.066,50.228,75.3
merge_yolov8n-p2_lsk_bifpn_DIoU_aug5_1280,0.853,0.763,0.517,0.724,3.066,50.228,78.4
merge_yolov8n-p2_lsk_bifpn_DIoU_aug10_1280,0.856,0.765,0.530,0.728,3.066,50.228,78.4


In [20]:
# 细节分析
df1 = get_size_map(path_base_yolov8n_1280)
df2 = get_size_map(path_proposed_1280)
improve_percent = ((df2-df1)/df1*100).round(1)

df2 = get_formatted_improve(df2, improve_percent)
pd.concat([df1.applymap(lambda x: f"{x:.3f}"), df2], axis=0).replace('-1.000', '-').replace('-1.000 (+00.0%)', '-')

,test_synth,test_real_nominal,test_real_edge,test_real
metrics,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.310,0.184,0.291
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.760,0.643,0.347,0.602
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.892,0.871,0.476,0.792
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.423 (+36.5%),0.408 (+121.7%),0.419 (+44.0%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.771 (+01.4%),0.693 (+07.8%),0.440 (+26.8%),0.661 (+09.8%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.894 (+00.2%),0.902 (+03.6%),0.621 (+30.5%),0.851 (+07.4%)


# baseline对比

In [21]:
metrics = []

# baseline_1280
for path in all_path_csv_baseline_1280:
    metrics.append(get_metrics(path))

# proposed_1280
metrics.append(get_metrics(path_proposed_1280))

# DF格式调整
metrics = set_metrics(metrics)

# 结果分析

# DF样式设计
display_metircs(metrics)

,test_synth,test_real_nominal,test_real_edge,test_real,param,flops,fps
models,,,,,,,
base_yolov8s_1280,0.865,0.757,0.439,0.703,11.136,114.588,83.9
merge_yolov8n-p2_lsk_bifpn_DIoU_aug10_1280,0.856,0.765,0.530,0.728,3.066,50.228,78.4


# Finetune
由于测试集不同 最后给出 用于最终优化

In [22]:
metrics = []

# finetune_yolov8n_1280
for path in all_path_csv_finetune:
    metrics.append(get_metrics(path))

# DF格式调整
metrics = pd.concat(metrics, axis=0)
metrics.rename(columns={'metrics': 'models'}, inplace=True)
metrics.set_index('models', inplace=True)

# 过滤打不过的模型
metrics = metrics[metrics.index != 'finetune_triple_split_yolov8n-p2_lsk_bifpn_DIoU_aug10_1280']

# 结果分析
metrics = metrics[['test_synth', 'test_real_nominal', 'test_real_edge', 'test_real']]
metrics.sort_values(by='test_real_edge', ascending=True, inplace=True)
metrics = metrics.replace(np.nan, '')
metrics

,test_synth,test_real_nominal,test_real_edge,test_real
models,,,,
finetune_single_yolov8n-p2_lsk_bifpn_DIoU_aug10_1280,0.83,0.717,0.477,0.678
finetune_double_yolov8n-p2_lsk_bifpn_DIoU_aug10_1280,,0.776,0.536,0.739
finetune_triple_yolov8n-p2_lsk_bifpn_DIoU_aug10_1280,,,0.727,
